In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine
import time
from datetime import timedelta
from datetime import datetime
from torchsummary import summary
import math
import os
import shutil
import csv

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs, CovidDataset, Rescale, ToTensor
from utils import averageModels, averageGradients
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Arguments():
    def __init__(self):
        self.images = 3012
        self.clients = 10
        self.rounds = 1001
        self.epochs = 1
        self.local_batches = 20
        self.lr = 0.01
        self.dropout1 = 0.25
        self.dropout2 = 0.5
        self.C = 0.66
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'noniid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = True
        self.save_model = False
        self.save_model_interval = 500
        self.clip = 1
        self.del_runs = False
        self.acc_csv = True
        self.acc_file = '0517_10clients_noniid1.csv'
        # number of classes per client on non iid case 
        self.noniid_classnum = 1
        # data transform
        self.transform = transforms.Compose([Rescale(32), ToTensor()])
        # number of classes
        self.c_num = 3

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


cuda


In [3]:
# delete files in runs (Tensorboard)
if args.del_runs==True:
    folder = 'runs'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [4]:
# create accuracy csv file
def acc_csv(args, rnd, acc):
    if args.acc_csv==True:
        with open("acc_csv_files/"+args.acc_file, 'a') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([rnd, acc])

In [5]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

print(clients)
print("number of clients : ", len(clients))

[{'hook': <VirtualWorker id:client1 #objects:0>}, {'hook': <VirtualWorker id:client2 #objects:0>}, {'hook': <VirtualWorker id:client3 #objects:0>}, {'hook': <VirtualWorker id:client4 #objects:0>}, {'hook': <VirtualWorker id:client5 #objects:0>}, {'hook': <VirtualWorker id:client6 #objects:0>}, {'hook': <VirtualWorker id:client7 #objects:0>}, {'hook': <VirtualWorker id:client8 #objects:0>}, {'hook': <VirtualWorker id:client9 #objects:0>}, {'hook': <VirtualWorker id:client10 #objects:0>}]
number of clients :  10


In [6]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, \
                                                                  args.iid, \
                                                                  args.transform, \
                                                                  args.c_num, \
                                                                  args.noniid_classnum)

(0,)
(1,)
(2,)
comb  [(0,), (1,), (2,)]
client_classes  [[0]
 [1]
 [2]
 [0]
 [1]
 [2]
 [0]
 [1]
 [2]
 [0]]
label count  Counter({0: 4, 1: 3, 2: 3})
users_dict  {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}
(0,)
(1,)
(2,)
comb  [(0,), (1,), (2,)]
client_classes  [[0]
 [1]
 [2]
 [0]
 [1]
 [2]
 [0]
 [1]
 [2]
 [0]]
label count  Counter({0: 4, 1: 3, 2: 3})
users_dict  {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}


In [7]:
print(len(global_train))
print(type(global_train))
print(len(global_test))
print(type(global_test))
print(len(train_group))
print(type(train_group))
print(len(test_group))
print(type(test_group))

3012
<class 'FLDataset.CovidDataset'>
753
<class 'FLDataset.CovidDataset'>
10
<class 'dict'>
10
<class 'dict'>


In [8]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    print("len(client", str(inx), "train set) = ", len(trainset_ind_list))
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
    client['samples'] = len(trainset_ind_list) / args.images

len(client 0 train set) =  251
len(client 1 train set) =  334
len(client 2 train set) =  334
len(client 3 train set) =  251
len(client 4 train set) =  335
len(client 5 train set) =  335
len(client 6 train set) =  251
len(client 7 train set) =  335
len(client 8 train set) =  335
len(client 9 train set) =  251


In [9]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = CovidDataset('./test.csv', transform=transforms.Compose([Rescale(32), ToTensor()]))
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True, drop_last=True)
print(len(global_test_loader))

37


In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                               out_channels = 32,
                               kernel_size = 3,
                               stride = 1)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1)
        self.fc1 = nn.Linear(14*14*64, 128)
#         self.fc1_bn = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.conv1_bn(x)
        x = F.relu(self.conv2(x))
        
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, p=args.dropout1)
        x = x.view(-1, 14*14*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=args.dropout2)
#         x = self.fc1_bn(x)
        x = self.fc2(x)
        return F.softmax(x)

In [11]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
#             output = client['model'](data.float())
#             loss = F.nll_loss(output, target.squeeze(1))
            output = client['model'](data)
            loss = client['criterion'](output, target.squeeze(1))
            loss.backward()
            
#             print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
#             print("output: ", output)
#             print("target squeeze: ", target.squeeze(1))
#             print("loss: ", loss)
#             print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            
#             print weight 
#             for name, param in client['model'].named_parameters():
#                 if name=='conv1_bn.weight':
#                     print(name, param.grad)
            
            torch.nn.utils.clip_grad_norm_(client['model'].parameters(), args.clip)
            client['optim'].step()
            
            
            if batch_idx % args.log_interval == 0 or batch_idx==len(client['trainset'])-1:
#                 loss = loss.get() 
                print('Model [{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, (batch_idx+1) * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * (batch_idx+1) / len(client['trainset']), loss.item()/args.log_interval))
                
#     client['model'].get() 

In [12]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for d in test_loader:
            data = d['image']
            target = d['label']
            data, target = data.to(device), target.to(device)
            if(str(device)=='cuda'):
                model.cuda()
            output = model(data.float())
#             test_loss += F.nll_loss(output, target.squeeze(1), reduction='sum').item() # sum up batch loss
            loss_fn = nn.CrossEntropyLoss(reduction='sum')
            test_loss += loss_fn(output, target.squeeze(1)).item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [13]:
writer = SummaryWriter()

In [14]:
torch.manual_seed(args.torch_seed)
global_model = Net().to(device)
summary(global_model, (1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             320
       BatchNorm2d-2           [-1, 32, 30, 30]              64
            Conv2d-3           [-1, 64, 28, 28]          18,496
            Linear-4                  [-1, 128]       1,605,760
            Linear-5                    [-1, 3]             387
Total params: 1,625,027
Trainable params: 1,625,027
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.82
Params size (MB): 6.20
Estimated Total Size (MB): 7.03
----------------------------------------------------------------


/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/native.py:414: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  response = command_method(*args_, **kwargs_)


In [15]:
# training
for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr, momentum = 0.8)
    client['criterion'] = nn.CrossEntropyLoss(reduction='mean')
#     client['pengine'] = PrivacyEngine(
#                                        client['model'],
#                                        batch_size=args.local_batches,
#                                        sample_size=len(client['trainset']),
#                                        alphas=range(2,32),
#                                        noise_multiplier=0,
#                                        max_grad_norm=1000
#                                     )
#     client['pengine'].attach(client['optim']) 
    
# start training model
training_start_time = time.time()
for fed_round in range(args.rounds):
    print("")
    print("===================================================================")
    print("[round] = ", fed_round+1, "/", args.rounds)
    print("===================================================================")
    
    round_train_start_time = time.time()
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))    
    
    # number of selected clients
    m = int(max(math.ceil(args.C * args.clients), 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
#     np.random.seed(fed_round)
#     active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
#     active_clients = [clients[i] for i in active_clients_inds]
    active_clients = selected_clients
    
    # Training 
    client_cnt = 0
    for client in active_clients:
        print("* [client count] = ", client_cnt+1 , "/", len(active_clients))
        client_train_start_time = time.time()
        ClientUpdate(args, device, client)
        client_cnt += 1
        client_train_time = round(time.time()-client_train_start_time)
        print("* [client_train_time] = ", str(timedelta(seconds=(client_train_time))))
        print("---------------------------------------------------------------")
    
#         # Testing 
#         for client in active_clients:
#             test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
#     print("active clients: ", active_clients)
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    acc = test(args, global_model, device, global_test_loader, 'Global')
    writer.add_scalar("Accuracy/train", acc, fed_round)
    writer.flush()
    acc_csv(args, fed_round, acc)
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
    # training time per round
    total_train_time = round(time.time()-training_start_time)
    round_train_time = round(time.time()-round_train_start_time)
    print("** [total train time]: ", str(timedelta(seconds=total_train_time)))
    print("** [round train time]: ", str(timedelta(seconds=round_train_time)))
    
    if (args.save_model and fed_round%args.save_model_interval==0 and fed_round!=0):
        now = datetime.now() 
        date = now.strftime("%Y_%m_%d_%H%M")
        torch.save(global_model.state_dict(), date + "_FedAvg_with_DP_round_" + str(fed_round) + ".pth")
        print("model saved : "+ date +"_FedAvg_with_DP_round_" + str(fed_round) + "10clients.pth")


/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/syft/frameworks/torch/hook/hook.py:560: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)



[round] =  1 / 1001
* [client count] =  1 / 7
Model [client3] Train Epoch: 1 [20/340 (6%)]	Loss: 0.111402
Model [client3] Train Epoch: 1 [220/340 (65%)]	Loss: 0.058258
Model [client3] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055508
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  2 / 7
Model [client9] Train Epoch: 1 [20/340 (6%)]	Loss: 0.114246
Model [client9] Train Epoch: 1 [220/340 (65%)]	Loss: 0.056957
Model [client9] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055363
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  3 / 7
Model [client5] Train Epoch: 1 [20/340 (6%)]	Loss: 0.111170
Model [client5] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055726
Model [client5] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055827
* [client_train_time] =  0:00:05
---------------------------------------------------------------
* [client count] =  4 / 7
Model [client10] Train Epoch:

Model [client2] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055198
Model [client2] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055257
* [client_train_time] =  0:00:05
---------------------------------------------------------------
* [client count] =  4 / 7
Model [client3] Train Epoch: 1 [20/340 (6%)]	Loss: 0.116667
Model [client3] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055284
Model [client3] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055159
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  5 / 7
Model [client10] Train Epoch: 1 [20/260 (8%)]	Loss: 0.104187
Model [client10] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055322
Model [client10] Train Epoch: 1 [260/260 (100%)]	Loss: 0.055264
* [client_train_time] =  0:00:01
---------------------------------------------------------------
* [client count] =  6 / 7
Model [client7] Train Epoch: 1 [20/260 (8%)]	Loss: 0.102801
Model [client7] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055536
Model [clie

Model [client6] Train Epoch: 1 [20/340 (6%)]	Loss: 0.125795
Model [client6] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055245
Model [client6] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055200
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  7 / 7
Model [client3] Train Epoch: 1 [20/340 (6%)]	Loss: 0.127068
Model [client3] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055223
Model [client3] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055185
* [client_train_time] =  0:00:06
---------------------------------------------------------------

Test set: Average loss for Global model: 0.8762, Accuracy: 495/753 (66%)

** [total train time]:  0:03:41
** [round train time]:  0:00:35

[round] =  8 / 1001
* [client count] =  1 / 7
Model [client9] Train Epoch: 1 [20/340 (6%)]	Loss: 0.075853
Model [client9] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055234
Model [client9] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055167
* [client_train_time] =  0:00:06
------

Model [client9] Train Epoch: 1 [20/340 (6%)]	Loss: 0.126673
Model [client9] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055199
Model [client9] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055215
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  2 / 7
Model [client3] Train Epoch: 1 [20/340 (6%)]	Loss: 0.129698
Model [client3] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055166
Model [client3] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055296
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  3 / 7
Model [client6] Train Epoch: 1 [20/340 (6%)]	Loss: 0.118416
Model [client6] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055526
Model [client6] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055208
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  4 / 7
Model [client7] Train Epoch: 1 [20/260 (8%)]	Loss: 0.071769
Model [client7] 

...

Model [client6] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055161
Model [client6] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055147
Model [client6] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055146
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  5 / 7
Model [client5] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055148
Model [client5] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055145
Model [client5] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:05
---------------------------------------------------------------
* [client count] =  6 / 7
Model [client2] Train Epoch: 1 [20/340 (6%)]	Loss: 0.057487
Model [client2] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055145
Model [client2] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:05
---------------------------------------------------------------
* [client count] =  7 / 7
Model [client8] Train Epoch: 1 [20/340 (6%)]	Loss: 0.056886
Model [client8] 

Model [client1] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055145
Model [client1] Train Epoch: 1 [260/260 (100%)]	Loss: 0.055226
* [client_train_time] =  0:00:01
---------------------------------------------------------------

Test set: Average loss for Global model: 0.6165, Accuracy: 681/753 (90%)

** [total train time]:  8:27:08
** [round train time]:  0:00:35

[round] =  990 / 1001
* [client count] =  1 / 7
Model [client8] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055146
Model [client8] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055145
Model [client8] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:05
---------------------------------------------------------------
* [client count] =  2 / 7
Model [client5] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055157
Model [client5] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055145
Model [client5] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055144
* [client_train_time] =  0:00:05
---------------------------------------------------------------


Model [client1] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055147
Model [client1] Train Epoch: 1 [260/260 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:01
---------------------------------------------------------------
* [client count] =  3 / 7
Model [client10] Train Epoch: 1 [20/260 (8%)]	Loss: 0.055148
Model [client10] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055145
Model [client10] Train Epoch: 1 [260/260 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:01
---------------------------------------------------------------
* [client count] =  4 / 7
Model [client4] Train Epoch: 1 [20/260 (8%)]	Loss: 0.055145
Model [client4] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055145
Model [client4] Train Epoch: 1 [260/260 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:01
---------------------------------------------------------------
* [client count] =  5 / 7
Model [client6] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055368
Model [client6] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055145
Model [clie

Model [client2] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055865
Model [client2] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055145
Model [client2] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055144
* [client_train_time] =  0:00:05
---------------------------------------------------------------
* [client count] =  6 / 7
Model [client4] Train Epoch: 1 [20/260 (8%)]	Loss: 0.055167
Model [client4] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055145
Model [client4] Train Epoch: 1 [260/260 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:01
---------------------------------------------------------------
* [client count] =  7 / 7
Model [client8] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055342
Model [client8] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055145
Model [client8] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:05
---------------------------------------------------------------

Test set: Average loss for Global model: 0.6617, Accuracy: 652/753 (87%)

** [total train time]:  8:3


Test set: Average loss for Global model: 0.6269, Accuracy: 677/753 (90%)

** [total train time]:  8:32:28
** [round train time]:  0:00:35

[round] =  1000 / 1001
* [client count] =  1 / 7
Model [client7] Train Epoch: 1 [20/260 (8%)]	Loss: 0.055145
Model [client7] Train Epoch: 1 [220/260 (85%)]	Loss: 0.055145
Model [client7] Train Epoch: 1 [260/260 (100%)]	Loss: 0.055153
* [client_train_time] =  0:00:01
---------------------------------------------------------------
* [client count] =  2 / 7
Model [client3] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055166
Model [client3] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055146
Model [client3] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055146
* [client_train_time] =  0:00:06
---------------------------------------------------------------
* [client count] =  3 / 7
Model [client9] Train Epoch: 1 [20/340 (6%)]	Loss: 0.055156
Model [client9] Train Epoch: 1 [220/340 (65%)]	Loss: 0.055146
Model [client9] Train Epoch: 1 [340/340 (100%)]	Loss: 0.055149
* [client_

In [16]:
# tensorboard open 
# tensorboard --logdir=/home/citi302/Desktop/Codefolder/FL_DP_covid/runs